In [1]:
import sys, os
import pandas as pd
sys.path.append("../../../../")

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient
db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

from Clust.clust.ML.common import ML_pipeline, tool
app_name= "energy" # "Hs2SwineFarmWithWeatherTime", "energy"

cpu is available.


## 1. Data Preparation

### 1-1. Parameter Setting

In [2]:
if app_name == "energy":
    model_purpose = 'regression'
    feature_X_list = ['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7',
       'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7',
       'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
    feature_y_list = ['value']
    split_mode ="windows_split"
    data_y_flag = True # 이미 만들어진 Y 데이터를 활용함
    
elif app_name == "Hs2SwineFarmWithWeatherTime":
    model_purpose = 'forecasting' 
    feature_X_list = ['Temperature', 'out_temp','sin_hour']
    feature_y_list = ['Temperature']
    split_mode = 'step_split'
    data_y_flag = False # Y데이터는 없음, X 에서 Y 데이터를 도출함
    
step = 'train'
bucket_name = 'integration' 
data_clean_level = 4

In [3]:
all_integrated_ms_list = db_client.measurement_list(bucket_name)
print(all_integrated_ms_list)
print("==========================================================")
collection_list = mongo_client.get_collection_list(bucket_name)
print(collection_list)

['classification_actionPattern_cleanLevel0_testX', 'classification_actionPattern_cleanLevel0_testy', 'classification_actionPattern_cleanLevel0_trainX', 'classification_actionPattern_cleanLevel0_trainy', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel0_trainX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_testX', 'forecasting_Hs2SwineFarmWithWeatherTime_cleanLevel4_trainX', 'forecasting_strawberryOpen_cleanLevel0_testX', 'forecasting_strawberryOpen_cleanLevel0_trainX', 'forecasting_strawberryOpen_cleanLevel4_testX', 'forecasting_strawberryOpen_cleanLevel4_trainX', 'regression_energy_cleanLevel0_testX', 'regression_energy_cleanLevel0_testy', 'regression_energy_cleanLevel0_trainX', 'regression_energy_cleanLevel0_trainy', 'regression_energy_cleanLevel4_testX', 'regression_energy_cleanLevel4_testy', 'regression_energy_cleanLevel4_trainX', 'regression_energy_cleanLevel4_trainy']
['forecasting_Hs2SwineFarmWithWeatherTime

### 1-2. Data Ingestion
#### 1-2-1. Select data name 

In [4]:
dataset_name = model_purpose + '_' + app_name  
data_name_X = dataset_name + '_cleanLevel' + str(data_clean_level)+'_'+step+'X'
data_name_y = dataset_name+'_cleanLevel' + str(data_clean_level)+'_'+ step+'y'
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'ms_name':data_name_X})[0]

In [5]:
data_meta

{'bucket_name': 'integration',
 'collection_name': 'regression_energy',
 'ms_name': 'regression_energy_cleanLevel4_trainX',
 'ingestion_type': 'multiple_ms_by_time',
 'ingestion_param': {'ms_list_info': [['life_indoor_environment',
    'humidityTrain_10min'],
   ['life_indoor_environment', 'temperatureTrain_10min'],
   ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']],
  'start_time': '2016-01-11',
  'end_time': '2016-04-15'},
 'processing_type': 'step_3',
 'process_param': {'refine_param': {'removeDuplication': {'flag': True},
   'staticFrequency': {'flag': True, 'frequency': None}},
  'outlier_param': {'certainErrorToNaN': {'flag': True},
   'unCertainErrorToNaN': {'flag': True,
    'param': {'outlierDetectorConfig': [{'algorithm': 'IQR',
       'percentile': 99,
       'alg_parameter': {'weight': 100}}]}}},
  'imputation_param': {'flag': True,
   'imputation_method': [{'min': 0,
     'max': 2,
     'method': 'linear',
     'parameter': {}}],
   'totalNonNanRatio

#### 1-2-2. X-y Data Ingestion pipeline

In [6]:
# Data Ingestion
ingestion_method = 'ms_all'
ingestion_param_X = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_X,
    'feature_list' : feature_X_list                              
}
ingestion_param_y = {
    "bucket_name" : bucket_name,
    'ms_name' : data_name_y,
    'feature_list' : feature_y_list                              
}

In [7]:
data_X, data_y = ML_pipeline.Xy_data_preparation(ingestion_param_X, data_y_flag, ingestion_param_y, ingestion_method, db_client)

In [8]:
print(len(data_X))
print(len(data_y))

13680
95


In [9]:
data_y

,value
time,
2016-01-11 00:00:00+00:00,19.379999
2016-01-12 00:00:00+00:00,12.680000
2016-01-13 00:00:00+00:00,5.340000
2016-01-14 00:00:00+00:00,12.720000
2016-01-15 00:00:00+00:00,13.250000
...,...
2016-04-10 00:00:00+00:00,14.710000
2016-04-11 00:00:00+00:00,13.690000
2016-04-12 00:00:00+00:00,13.870000


### 1-2-2. Random Nan Insert (Test)

In [10]:
nan_ratio = 0.00

In [11]:
data_X = tool.random_nan_df(data_X, nan_ratio)
data_y = tool.random_nan_df(data_y, nan_ratio)

../../../../Clust/clust/ML/common/tool.py:17: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead.
  df.loc[df.sample(frac=nan_ratio). index, col] = pd.np.nan


#### 1-2-3. Data scaling

In [12]:
scaler_param='scale'
scale_method='minmax'
scaler_path = './scaler/'

In [13]:
dataX_scaled, X_scalerFilePath, datay_scaled, y_scalerFilePath= ML_pipeline.Xy_data_scaling_train(data_name_X, data_X, data_name_y, data_y, scaler_path, scaler_param, scale_method)

['Press_mm_hg', 'RH_1', 'RH_2', 'RH_3', 'RH_4', 'RH_5', 'RH_6', 'RH_7', 'RH_8', 'RH_9', 'RH_out', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T_out', 'Tdewpoint', 'Visibility', 'Windspeed']
Make New scaler File
['value']
Make New scaler File


## 2. Cleaning and split
### 2.1 pipeline - clean low quality column

In [14]:
model_clean = False
nan_process_info = {'type':'num', 'ConsecutiveNanLimit':10000, 'totalNaNLimit':100000}

In [15]:
dataX_scaled = ML_pipeline.clean_low_quality_column(model_clean, nan_process_info, dataX_scaled)
feature_X_list= list(dataX_scaled.columns)

### 2.2 Train/Val Split pipeline

In [16]:
split_ratio = 0.8

In [17]:
# TODO 데이터 나뉘는 부분 추가로 작성된 것 지수님에게 물어봐야 함
day_window_size = tool.get_default_day_window_size(dataX_scaled)
train_x, val_x, train_y, val_y = ML_pipeline.split_data_by_mode(split_mode, split_ratio, dataX_scaled, datay_scaled, day_window_size)

### 2.3 Data Transformation & Clean2 pipeline

In [18]:
max_nan_limit_ratio = 0.5
if split_mode =='windows_split':
    transformParameter = {
            'past_step':day_window_size,
            'max_nan_limit_ratio': max_nan_limit_ratio
    }
else:
    transformParameter = {
            'future_step': 2,
            'past_step': 12, 
            'max_nan_limit_ratio': max_nan_limit_ratio
    }

In [19]:
train_X_array, train_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, train_x, train_y)
val_X_array, val_y_array = ML_pipeline.transform_data_by_split_mode(split_mode, transformParameter, val_x, val_y)

window_size: 144 nan_limit_num: 72
(10944, 24) (72, 144, 24)
(76, 1) (72, 1)
window_size: 144 nan_limit_num: 72
(2736, 24) (19, 144, 24)
(19, 1) (19, 1)


In [20]:
print(train_X_array.shape)
print(train_y_array.shape)
print(val_X_array.shape)
print(val_y_array.shape)

(72, 144, 24)
(72, 1)
(19, 144, 24)
(19, 1)


### 2.4 Set Model and train parameter

In [21]:
# RNN models (RNN, LSTM, GRU) parameters
seq_len, input_size = train_X_array.shape[1], train_X_array.shape[2]
model_method = 'GRU_rg' # Set model methods i.e., 'LSTM_rg', 'GRU_rg', 'CNN_1D_rg', 'LSTM_FCNs_rg', 'FC_rg' 
if model_method == 'LSTM_rg' or model_method == 'GRU_rg':
    modelParameter = {
        'rnn_type': 'lstm',
        'input_size': input_size, 
        'hidden_size': 64,
        'num_layers': 2,
        'output_dim': 1, 
        'dropout': 0.1, 
        'bidirectional': True
    }
# CNN_1D model parameters
elif model_method == 'CNN_1D_rg':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'output_channels': 64,
    'kernel_size': 3,
    'stride': 1,
    'padding': 0, 
    'dropout': 0.1
    }
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_rg':
    modelParameter = {
    'input_size': input_size,
    'num_layers': 2,
    'lstm_dropout': 0.4,
    'fc_dropout': 0.1
    }
# FC model parameters
elif model_method == 'FC_rg':
    modelParameter = {
    'input_size': input_size,
    'dropout': 0.1,
    'bias': True
    }
    
train_parameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 100, 
    'batch_size': 16
}
model_name = None
model_file_path = None

### 2.5 Set Model name and path pipeline

In [22]:
# model_name and file path check and create
model_name = tool.get_default_model_name(model_name, app_name, model_method, model_clean)
model_file_path = tool.get_default_model_path(model_name, data_name_X, model_method, train_parameter)

./Models/GRU_rg/energy_GRU_rg_False/regression_energy_cleanLevel4_trainX/c6f2ca56e27be1937ecf1e173440959e/model.pkl


In [23]:
model_name

'energy_GRU_rg_False'

## 3. Training

In [24]:
ML_pipeline.CLUST_regresstion_train(train_parameter, model_method, modelParameter, model_file_path, train_X_array, train_y_array, val_X_array, val_y_array)

Start training model
[1/100] Training loss: 0.2152	 Validation loss: 0.1540
[2/100] Training loss: 0.1981	 Validation loss: 0.1418
[3/100] Training loss: 0.1745	 Validation loss: 0.1300
[4/100] Training loss: 0.1777	 Validation loss: 0.1185
[5/100] Training loss: 0.1632	 Validation loss: 0.1072
[6/100] Training loss: 0.1450	 Validation loss: 0.0962
[7/100] Training loss: 0.1329	 Validation loss: 0.0855
[8/100] Training loss: 0.1143	 Validation loss: 0.0751
[9/100] Training loss: 0.0915	 Validation loss: 0.0653
[10/100] Training loss: 0.0948	 Validation loss: 0.0561
[50/100] Training loss: 0.0379	 Validation loss: 0.0331
[100/100] Training loss: 0.0346	 Validation loss: 0.0332

Training complete in 0m 11s


## 4. save metadata

In [25]:
# from Clust.clust.transformation.general.dataScaler import encode_hash_style
modelTags =["model_tag_example"]
trainDataType = "timeseries"

modelInfoMeta ={
    "trainDataInfo":data_meta,
    "modelName":model_name,
    "dataSplitMode":split_mode,
    "featureXList":feature_X_list,
    "featureyList":feature_y_list,
    "data_y_flag": data_y_flag,
    "trainDataType":trainDataType,
    "modelPurpose":model_purpose,
    "modelMethod":model_method,
    "modelTags":modelTags,
    "modelCleanLevel":model_clean,
    "trainParameter": train_parameter,
    "modelParameter": modelParameter,
    "transformParameter":transformParameter,
    "scalerParam":scaler_param,
    "trainDataName":[data_name_X, data_name_y], 

    "files":{
        "modelFile":{
            "fileName":"model.pth",
            "filePath":model_file_path
        },
        "XScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":X_scalerFilePath       
        },
        "yScalerFile":{
            "fileName":"scaler.pkl",
            "filePath":y_scalerFilePath      
        }
    }
}

In [26]:
from Clust.clust.ML.tool import meta as ml_meta
modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, modelInfoMeta)

========== Data Save Success ==========
{'trainDataInfo': {'bucket_name': 'integration', 'collection_name': 'regression_energy', 'ms_name': 'regression_energy_cleanLevel4_trainX', 'ingestion_type': 'multiple_ms_by_time', 'ingestion_param': {'ms_list_info': [['life_indoor_environment', 'humidityTrain_10min'], ['life_indoor_environment', 'temperatureTrain_10min'], ['weather_outdoor_environment', 'belgiumChieverseAirportTrain_10min']], 'start_time': '2016-01-11', 'end_time': '2016-04-15'}, 'processing_type': 'step_3', 'process_param': {'refine_param': {'removeDuplication': {'flag': True}, 'staticFrequency': {'flag': True, 'frequency': None}}, 'outlier_param': {'certainErrorToNaN': {'flag': True}, 'unCertainErrorToNaN': {'flag': True, 'param': {'outlierDetectorConfig': [{'algorithm': 'IQR', 'percentile': 99, 'alg_parameter': {'weight': 100}}]}}}, 'imputation_param': {'flag': True, 'imputation_method': [{'min': 0, 'max': 2, 'method': 'linear', 'parameter': {}}], 'totalNonNanRatio': 90}}, 'i